<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/04-다중_언어_개체명_인식.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  텍스트 분류 목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 데이터셋](#chapter1)
* [Chapter 2 다중 언어 트랜스포머](#chapter2)
* [Chapter 3 XLM-R 토큰화](#chapter3)
    * [Chapter 3-1 토큰화 파이프라인](#chapter3-1)
    * [Chapter 3-2 SentencePiece 토크나이저](#chapter3-2)
* [Chapter 4 개체명 인식을 위한 트랜스포머](#chapter4)    
* [Chapter 5 트랜스포머 모델 클래스](#chapter5)    
    * [Chapter 5-1 토큰 분류를 위한 사용자 정의 모델 만들기](#chapter5-1)    
    * [Chapter 5-2 사용자 정의 모델 로드하기](#chapter5-2)    

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. NLP를 적용할 문서가 다른 언어이거나, 다국어로 되어있는 경우 문제가 발생할 수 있다.
    - 러시아어, 중국어, 독일어와 같은 대표적인 언어는 허깅페이스에서 적절한 사전 훈련된 언어 모델을 찾아 미세튜닝 가능하다.
    - 그리스어, 스와힐리어 등의 언어는 사전 훈련된 모델을 찾기 어려울 수 있다.

2. 다중 언어 트랜스포머 모델이 등장하면서, 하나의 모델로 여러 언어를 처리할 수 있게 되었다.
    - 대표적인 다중 언어 트랜스포머 모델: mBERT, XLM-RoBERTa
    - 다중 언어 트랜스포머 모델은 여러 언어로 된 대규모 말뭉치를 사용해 사전 훈련되었다.
    - 다중 언어 트랜스포머 모델은 단일 언어 모델에 비해 성능이 약간 떨어질 수 있지만, 다양한 언어를 처리할 수 있다는 장점이 있다.

3. 다중 언어 트랜스포머는 많은 언어로 된 대규모 말뭉치에서 사전 훈련해서 제로샷 교차 언어 전이가 가능하다.
    - 제로샷 교차 언어 전이: 모델이 훈련되지 않은 언어로 된 문서에 대해 예측할 수 있는 능력
    - 예를 들어, 영어로 된 데이터셋으로 미세튜닝한 모델이 러시아어로 된 문서에 대해서도 예측할 수 있다.

4. 이번 장에서는 XML-RoBERTa 모델을 사용해 다중 언어 개체명 인식(NER) 작업을 수행한다.
    - 개체명 인식(NER): 문서에서 사람, 장소, 조직 등의 개체를 식별하는 작업
    - 다중 언어 NER 작업에서는 여러 언어로 된 문서에서 개체를 식별해야 한다.
    - 예를 들어 회사 문서에서 중요한 정보를 추출하거나, 검색 엔진의 품직을 높이거나, 말뭉치에서 구조적인 데이터를 생성하는 데 활용할 수 있다.

5. 네 개의 공용어를 사용하여 스위스에서 주로 활동하는 고객을 위해 NER을 수행한다고 가정한다.

## Chapter 1 데이터셋 <a class="anchor" id="chapter1"></a>
1. WikiANN 또는 APN-X라 불리는 교차 언어 전이 평가 벤치마크 데이터를 사용한다.
    - 여러 언어의 위키피디피아 문서로 구성되고 스위스에서 사용되는 독일어(62.9%), 프랑스어(22.9%), 이탈리아어(8.4%), 영어(5.9%) 네 개 언어로 작성됐다.
    - 각 문서는 IOB2 포멧으로 LOC(위치), PER(사람), ORG(조직) 세 가지 개체 유형으로 주석이 달려 있다.
        - B- 접두사: 개체명의 시작을 나타낸다.
        - I- 접두사: 동일한 개체명에 속해 연속되는 토큰을 나타낸다.
        - O: 개체명에 속하지 않는 토큰을 나타낸다.
    - 예를 그림으로 표현하면 아래와 같다.

         ![WikiANN 예시](image/04_01_WikiANN.png)


In [2]:
# TMREME에서 PAN-X 서브셋 중 하나를 로드한다.
#   - load_dataset() 함수에 전달할 이름을 확인하기 위해 get_dataset_config_names() 함수를 사용한다.
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME 서브셋 개수: {len(xtreme_subsets)}")
print(xtreme_subsets)

XTREME 서브셋 개수: 183
['MLQA.ar.ar', 'MLQA.ar.de', 'MLQA.ar.en', 'MLQA.ar.es', 'MLQA.ar.hi', 'MLQA.ar.vi', 'MLQA.ar.zh', 'MLQA.de.ar', 'MLQA.de.de', 'MLQA.de.en', 'MLQA.de.es', 'MLQA.de.hi', 'MLQA.de.vi', 'MLQA.de.zh', 'MLQA.en.ar', 'MLQA.en.de', 'MLQA.en.en', 'MLQA.en.es', 'MLQA.en.hi', 'MLQA.en.vi', 'MLQA.en.zh', 'MLQA.es.ar', 'MLQA.es.de', 'MLQA.es.en', 'MLQA.es.es', 'MLQA.es.hi', 'MLQA.es.vi', 'MLQA.es.zh', 'MLQA.hi.ar', 'MLQA.hi.de', 'MLQA.hi.en', 'MLQA.hi.es', 'MLQA.hi.hi', 'MLQA.hi.vi', 'MLQA.hi.zh', 'MLQA.vi.ar', 'MLQA.vi.de', 'MLQA.vi.en', 'MLQA.vi.es', 'MLQA.vi.hi', 'MLQA.vi.vi', 'MLQA.vi.zh', 'MLQA.zh.ar', 'MLQA.zh.de', 'MLQA.zh.en', 'MLQA.zh.es', 'MLQA.zh.hi', 'MLQA.zh.vi', 'MLQA.zh.zh', 'PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X

In [3]:
# "PAN"으로 시작하는 서브셋을 찾는다.
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
print(f"PAN-X 서브셋 개수: {len(panx_subsets)}")
print(panx_subsets)

PAN-X 서브셋 개수: 40
['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl', 'PAN-X.pt', 'PAN-X.ru', 'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th', 'PAN-X.tl', 'PAN-X.tr', 'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh']


2. ISO 639-1 언어 코드로 보이는 두 문자로 된 접미사가 있다.
    - de: 독일어
    - fr: 프랑스어
    - it: 이탈리아어
    - en: 영어

In [5]:
from datasets import load_dataset
panx_de = load_dataset("xtreme", "PAN-X.de")
print(panx_de)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})


3. 독일어, 프랑스어, 이탈리아어, 영어 말뭉치를 샘를링한다.
    - 분균형한 데이터셋이 만들어지는데, 실제 데이터셋에서는 흔히 벌어지는 일이다.
    - 소수 언어에서 레이블링된 샘플을 구하려면 비용이 많이 들기도 한다. 

In [6]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]  # PAN-X 언어별 샘플 비율

panx_ch = defaultdict(DatasetDict) # 키가 없는 경우 빈 DatasetDict 생성

for lang, frac in zip(langs, fracs):
    # 다국어 말뭉치를 로드합니다.
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # 각 분할을 언어 비율에 따라 다운샘플링하고 섞습니다.
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))

In [12]:
# 훈련 세트에 언어마다 얼마나 많은 샘플이 들어있는지 확인한다.
import pandas as pd

pd.DataFrame({lang: panx_ch[lang]["train"].num_rows for lang in langs}, index=["훈련 샘플 수"])

,de,fr,it,en
훈련 샘플 수,12580,4580,1680,1180


4. 독일어 샘플은 그 외 언어를 모두 합친 것보다 더 많다.

5. 이 데이터셋을 사용해 제로샷 교차 언어 전이를 프랑스어, 이탈리아어, 영어에 수행한다.

In [13]:
# 독일어 말뭉치에 있는 샘플 한 개 확인
#   - 샘플의 키는 애로우 테이블의 열 이름에 해당하고 값은 각 열에 있는 항목이다.
#   - ner_tags 열은 각 개체면에 매핑된 클래스 ID에 해당한다.
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [ ]:
# 이해하기 쉽게 LOC, PER, ORG 태크로 새로운 열을 만든다.
#   - Dataset 객체는 각 열의 데이터 타입을 담은 feature 속성을 가진다.
#   - nert_tags 열은 ClassLable의 리스트이다.
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: List(Value('string'))
ner_tags: List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))
langs: List(Value('string'))


In [7]:
# 훈련 세트에서 특성을 확인한다
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])


In [19]:
# int2str() 메서드를 사용해 클래스 ID를 레이블 문자열로 변환한다.
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(i) for i in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names, batched=True)

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

In [20]:
# 첫 번째 샘플의 토큰과 태그 이름을 나란히 출력한다.
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]], index=["토큰", "태그"])

,0,1,2,3,4,5,6,7,8,9,10,11
토큰,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
태그,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [ ]:
# 태그가 분균형하게 부여되지 않았나 확인하기 위해 각 분할에서 개체면의 빈도를 계산한다.
#  - B-LOC, B-PER, B-ORG 태그로 시작하는 태그의 개수를 센다.
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index") 

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


## Chapter 2 다중 언어 트랜스포머 <a class="anchor" id="chapter2"></a>
1. 다중 언어 트랜스포머의 훈련 과정과 아키텍쳐는 단일 언어 트랜스포머와 비슷하다.
    - 다중 언어 트랜스포머는 여러 언어로 된 대규모 말뭉치를 사용해 사전 훈련된다.
    - 예를 들어, mBERT는 104개 언어로 된 위키피디피아 문서를 사용해 사전 훈련되었다.
    - XLM-RoBERTa는 100개 언어로 된 CommonCrawl 데이터를 사용해 사전 훈련되었다.

2. NER에 대한 교차 언어 전이의 과정을 측정하기 위해 CoNLL-2002와 CoNLL-2003 데이셋이 영어, 네덜란드어, 스페인어, 독일어를 위한 벤치 마크로 자주 사용된다.
    - 이 벤치마크틑 PAN-X와 동일하게 개체명이 LOC, PER, ORG 태그로 분류된 뉴스 기사로 구성된다.
    - 세 카타고리에 속하지 않은 개체명을 위해 MISC 태그도 포함된다.

3. 다중 언어 트랜스포머 모델은 일반적으로 세 가지 방식으로 평가한다.
    - en: 영어 훈련 데이터에서 미세 튜닝한 다음에 각 언어의 테스트 세트에서 평가한다.
    - each: 언어별 성능을 측정하기 위해 단일 언어의 테스트 세트에서 미세 튜닝하고 평가한다.
    - all: 모든 언어의 훈련 데이터를 결합해 미세 튜닝한 다음에 각 언어의 테스트 세트에서 평가한다.

4. NER 평가 모델로 XLM-RoBERTa 모델을 사용한다.
    - XLM-RoBERTa는 사전 훈련 방식이 단일 언어 모델 RoBERTa와 동일한 다중 언어 트랜스포머 모델이다.
    - 100개 언어로 된 CommonCrawl 데이터를 사용해 사전 훈련되었다.
    - XLM에서 사용하는 언어 임베딩을 사용하지 않고, 단일 어휘 집합을 사용해 토큰화한다.
        - 따라서 동일한 단어의 토큰은 동일한 임베딩 ID를 갖는다. 

## Chapter 3 XLM-R 토큰화 <a class="anchor" id="chapter3"></a>
1. XLM-R은 WordPice 토크나이져 대신 100개의 언어 텍스트에서 훈련된 SentencePiece 토크나이져를 사용한다.

In [12]:
# SentencePice 토크나이져와 WordPiece 토크나이져 비교를 위해 토크나이져 로드
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased" # BERT 모델 이름
xlmr_model_name = "xlm-roberta-base" # XLM-R 모델 이름

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name) # BERT 토크나이져
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name) # XLM-R 토크나이져

loading configuration file config.json from cache at /home/redinblue/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /home/redinblue/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/vocab.txt
loading file tokenizer.json from cache 

In [5]:
# 짧은 텍스트 시퀸스를 인코딩해서 각 모델이 사전 훈련 동안에 사용하는 특수 토큰 확인
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
print(bert_tokens)
xlmr_tokens = xlmr_tokenizer(text).tokens()
print(xlmr_tokens)

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']
['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']


### Chapter 3-1 토큰화 파이프라인 <a class="anchor" id="chapter3-1"></a>
1. 토큰화는 아래의 이미지와 같은 네 단계의 파이프라인으로 구성된다.

     ![XLM-R 토큰화 파이프라인](image/04_03_XLMR_Tokenization_Pipeline.png)

2. 정규화
     - 원시 문자역을 더 "깨긋하게" 만들기 위해 적용하는 일련의 연산이다.
     - 예를 들어, 대문자를 소문자로 변환하거나, 악센트 부호를 제거하거나, 특수 문자를 처리하는 작업이 포함된다.
     - 유니코드 정규화는 많은 토크나이저에서 적용되는 또 다른 일반적인 정규화 연산이며, 같은 문자를 쓰는 여러 가지 방식을 처리한다.
          - 같은 문자열의 두 버전이 다르게 표시될 수도 있다.
          - 예를 들어, "é" 문자는 단일 문자로 표현될 수도 있고, "e" 문자와 악센트 부호로 분리되어 표현될 수도 있다.

3. 사전 토큰화
     - 텍스트를 더 작은 객체로 분활하며 훈련 마지막에 생성되는 토큰의 상한선을 제공한다.
     - 사전 토큰화가 텍스트를 단어로 분할하고 최종 토큰은 이 단어의 일부가 되다.
     - 예를 들어, "Jack Sparrow loves New York!" 문장은 공백과 특수문자를 기준으로 분할되어 여섯 개로 나뉜다.
          - ["Jack", "Sparrow", "loves", "New", "York", "!"]
     - 이 단어들은 다음단계에서 BPE나 유니그램 알고리즘을 상용해서 더 작은 단위로 분할된다.

4. 토크나이져 모델
     - 입력 텍스트 정규화와 사전 토큰화를 수행하고 난 후 토크나이져를 사용해 부분단어 분할 모델을 단어에 적용한다.
     - 토크나이져는 파이프라인에서 말뭉치로 훈련이 필요한 부분이다.
     - 단어를 부분단어로 나눠 어휘사전의 크기와 OOV 토큰의 개수를 출이는 역활을 한다.
     - 예시 문장에 토크나이져 모델을 적용하면 다음과 같은 토큰이 생성된다.
          - ["jack", "spa", "rrow", "love", "s", "new", "york", "!"]
     - 이 시점부터 더 이상 문자열 리스트가 아니라 정수(입력ID) 리스트를 가지게 된다.

5. 사후 처리
     - 토큰 리스트에 부가적인 변환을 적용한다.
     - 예를 들어, 모델에 입력하기 위해 특수 토큰을 추가하거나, 토큰 리스트를 고정된 길이로 패딩하는 작업이 포함된다.
     - BERT 모델에서는 [CLS] 토큰을 문장 맨 앞에 추가하고, [SEP] 토큰을 문장 맨 뒤에 추가한다.
     - XLM-R 모델에서는 <s> 토큰을 문장 맨 앞에 추가하고, </s> 토큰을 문장 맨 뒤에 추가한다.




### Chapter 3-2 SentencePiece 토크나이저 <a class="anchor" id="chapter3-2"></a>
1. SentencePiece 토크나이저는 유니그램이라는 부분단어 분할 방식을 기반으로 각 입력 텍스트를 유니코드 문자 시퀸스로 인코딩한다.
    - 악센트, 구두점에 대해 몰라도 되므로 이 특징은 다국어 말뭉치에 특히 유용하다.

2. 공백 문자가 유니코드 기호 U+2581또는 1/4 블록 문자라고 하는 "_"문자에 할당된다.
    - 언어별 사전 토크나이저에 의종하지 않고 정확하게 시퀸스를 복원한다.
    - WordPice는 "York"와 "!"사이에 공백이 없다는 정보를 잃어버리린다.
    - SenntencePiece는 토큰화된 텍스트에 공백을 보존하기 때문에 정확하게 원시 텍스트로 다시 변환한다.

In [7]:
"".join(xlmr_tokens).replace(u"\u2581"," ")

'<s> Jack Sparrow loves New York!</s>'

3. 간단한 샘플을 NER에 적합한 형태로 인코딩하는 과정을 살펴본다.
    - 토큰 분류 헤드와 함께 사전 훈련되 모델을 로드한다.
    - 헤드를 직접 만들 수 있다.

## Chapter 4 개체명 인식을 위한 트랜스포머 <a class="anchor" id="chapter4"></a>
1. 텍스트 분류를 위해 BERT는 특수 토큰 [CLS]로 전체 텍스트 시퀸스를 표현한다.
    - 이 표현을 완전 연결 또는 밀집 층에 통화시켜 이산적인 레이블 값을 출력한다.

         ![텍스트 분류를 위한 BERT](image/04_04_BERT_Text_Classification.png)

2. BERT와 그 외 인코더 기반 트랜스포머는 NER 작업에 비슷한 방식을 사용한다.
    - 다만 모든 입력 토큰의 표현이 완전 연결 층에 주입되어 해당 토큰의 개체명을 인식한다.
    - NER를 종종 토큰 분류 작업으로 생각하기도한다.

        ![개체명 인식을 위한 BERT](image/04_05_BERT_Named_Entity_Recognition.png)

3. 토큰 분류 작업에서 부분단어는 어떻게 처리할까?
    - 위의 그림에서 "Christa"는 부분단어 "Chr"와 "##ista"로 토큰화됐다.
    - 이중 어는 단어 아니면 두 단어 모두에 B-PER 레이블을 할당해야 하는가?
    - BERT 논문에서 저잗들은 이 레이블을 첫 번때 부분어("Chr")에 할당하고 이어지는 부분단어("##ista")는 무시했다.
        - 무시한 부분단어는 ING로 표시한다.
    - 후처리 단계에서 첫 번째 부분단어의 예측 레이블을 후손 부분단어로 쉽게 전파할 수 있다.         


## Chapter 5 트랜스포머 모델 클래스 <a class="anchor" id="chapter5"></a>
1. 트랜스포머스는 아키텍처와 작업마다 전용 클래스를 제공한다.
    - 작업에 연관된 모델 클래스 이름은 <ModelNAme>For<task> 형식을 따른다.
    - 예를 들어, XLM-RoBERTa 모델을 사용해 토큰 분류 작업을 수행하려면 XLMRobertaForTokenClassification 클래스를 사용한다.

2. 트랜스포머 API를 자세히 파악하기 위해 다음 시나리오을 살펴본다.
    - 트랜스포머 모델로 오랫동안 생각해온 NLP 문제를 해결할 좋은 아이디어가 떠올랐다고 가정한다.

3. 트랜스포머스의 다재다능한 능력은 바디와 헤드로 나뉜 모델 구조에서 나온다.
    - 모델의 마지막 층이 후속 작업에 맞는 층으로 바뀌는데, 이 마지막 층이 모델 헤드이다.
    - 나머지 층이 모델 바디며, 여기에는 작업에 특화되지 않은 토큰 임베딩과 트랜스포머 층이 포함된다.

4. 이 구조는 트랜스포머스 코드에도 반영된다.
    - 모델의 바디는 BertModel 모델 또는 GPT2Model 같은 클래스로 구현되며 마지막 층의 은닉 상태를 반환한다.
    - BertForMaskedLM 또는 BertForSequenceClassification 같은 클래스는 모델 바디를 상속받아 작업에 특화된 헤드를 추가한다.
    - BertModel 클래스는 모델의 바디만 포함이고, BertFor<Task> 클래스는 바이와 작업 전용 헤드를 모두 포함한다.

         ![트랜스포머 모델 바디와 헤드](image/04_06_Transformer_Model_Body_and_Head.png) 

5. 바디와 헤드가 분리된 구조 덕분에 특정 작업을 위해 만든 사용저 정의 헤드를 사전 훈련된 모델 바디에 쉽게 연결할 수 있다.
    - 예를 들어, BERT 모델 바디에 사용자 정의 NER 헤드를 연결할 수 있다.
    - 트랜스포머스는 모델 바디와 헤드를 결합하는 데 도움이 되는 여러 가지 유틸리티 함수를 제공한다.


### Chapter 5-1 토큰 분류를 위한 사용자 정의 모델 만들기 <a class="anchor" id="chapter5-1"></a>
1. XLM-R에 대한 사용자 정의 토큰 분류 헤드를 만드는 예제를 다룬다.
    - XLM-R은 RoBERTa와 모델 구조가 동일하기 때문에 RoBERTa를 베이스 모델로 사용하여 특화된 설정을 추가한다.
    - 자신의 작업에 맞는 모델을 만드는 방법을 이해하는 데 도움이 된다.
    - XLM-R NER 태그를 표현할 데이터 구조가 필요하다.
    - 모델을 초기화할 설정객체와 출력을 생성할 forward() 함수가 필요하다.

2. 간단한 클래스에 두 개의 함수를 구현하여 사용자 정의 트랜스포머 모델을 만든다.
    - RobertaPreTrainedModel를 상속했으므로 from_pretrained() 메서드와 같은 유용한 유틸리티가 모두 사용가능하다.


In [ ]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    # config_class는 RobertaPreTrainedModel의 클래스 속성으로, 모델을 초기화할 때 사용할 설정 클래스를 지정한다.
    config_class = XLMRobertaConfig
    
    def __init__(self, config):
        # 부모클래스를 로드하여 기본 설정을 초기화한다.
        super().__init__(config)
        self.num_labels = config.num_labels
        # 모델 바디를 로드한다.
        #   - add_pooling_layer=False로 설정하여 [CLS]를 제외하고 모든 토큰에 대한 출력을 얻는다. 
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # 토큰 분류 헤드를 정의한다.
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # 가중치를 로드하고 초기화한다.
        #   - RobertaPreTrainedModel에 정의되어 있으며 post_init() 메서드를 호출하는 것이 더 좋다.
        self.init_weights()
        
    def forward(
        self,
        input_ids=None,  
        attention_mask=None, 
        token_type_ids=None, 
        labels=None,
        **kwargs):# 추가적인 매개변수 무시
        ''' 정방향 패스에서 모델이 할 일을 정의한다.'''
        
        # 모델 바디를 사용해 인코더 표현을 얻는다.
        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, **kwargs)
        # 인코더 표현을 헤드에 통과시킨다.
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # 손실을 계산한다.
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # logits.view(-1, self.num_labels)과 labels.view(-1)을 사용해 2D 텐서로 변환
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        #  모델 출력 객체를 반환한다.
        #   - 네임드 튜플로 원소를 참조할 수 있도록 TokenClassifierOutput을 사용
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


### Chapter 5-2 사용자 정의 모델 로드하기   <a class="anchor" id="chapter5-2"></a>
1. 각 개체명을 레이블링하는 데 사용할 태크, 각 태크를 ID로 매핑하는 딕셔너리와 역매핑 딕셔너리가 필요하다.
    - NER 태그: ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
    - 태그를 ID로 매핑하는 딕셔너리: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4, 'B-ORG': 5, 'I-ORG': 6}
    - ID를 태그로 매핑하는 딕셔너리: {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-LOC', 4: 'I-LOC', 5: 'B-ORG', 6: 'I-ORG'}

In [9]:
index2tag = {idx: tag for tag, idx in enumerate(tags.names)}
print(index2tag)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [10]:
tag2index = {tag: idx for idx, tag in index2tag.items()}
print(tag2index)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}


2. AutoConfig 객체에 이런 매핑과 tag.num_classes 속성을 저장한다.
    - AutoConfig 클래스는 모델 구조의 청사진을 가진다.
    - AutoModel.from_pretrained()로 모델을 로드할 때 모델에 연관된 설정이 자동으로 다운된다.
        - xlmr_config를 따로 전달하지 않으면 XLM-R 모델의 기본 설정이 사용된다.

In [13]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(xlmr_model_name, 
                                         num_labels=tags.num_classes,
                                         id2label=index2tag,
                                         label2id=tag2index)

loading configuration file config.json from cache at /home/redinblue/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/e73636d4f797dec63c3081bb6ed5c7b0bb3f2089/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "B-LOC": 5,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type"

3. config 매겨 변수를 추가한 다음, 이전 처럼 from_pretrained() 메서드를 사용해 모델 가중치를 로드한다.

In [14]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, config=xlmr_config)
              .to(device))

Using device: cuda


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /home/redinblue/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/e73636d4f797dec63c3081bb6ed5c7b0bb3f2089/model.safetensors
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights 

In [18]:
import pandas as pd
# 토크나이저와 모델을 바르게 초기화했는지 빠르게 확인하기 위해 개체명을 알고 있는 문장을 입력으로 사용한다.
#   - 텍스트를 입력 ID로 전환한다.
#   - encode() 메서드는 토크나이저를 호출했을 때 반환되는 딕셔너리 중 input_ids 키에 해당하는 값을 반환한다.
text = "Jack Sparrow loves New York!"
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
print(f"입력 ID: {input_ids}")

xlmr_tokens = xlmr_tokenizer(text).tokens()
print(f"토큰: {xlmr_tokens}")

pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["토큰", "입력 ID"])

입력 ID: tensor([[    0, 21763, 37456, 15555,  5161,     7,  2356,  5753,    38,     2]])
토큰: ['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']


,0,1,2,3,4,5,6,7,8,9
토큰,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
입력 ID,0,21763,37456,15555,5161,7,2356,5753,38,2


In [ ]:
# 모델에 입력 ID를 전달하고 argmax 함수로 토큰마다 확률이 가장 높은 클래스를 선택해 예측을 만든다.
outputs = xlmr_model(input_ids.to(device)).logits
predictions = outputs.argmax(dim=-1)
print(f"시킨스에 있는 토킁의 개수: {len(xlmr_tokens)}")

# 로짓의 크기는 [배치 크기, 시퀸스 길이, 클래스 수]이다.
print(f"출력 크기: {outputs.shape}")
print(f"예측된 클래스 ID: {predictions}")

시킨스에 있는 토킁의 개수: 10
출력 크기: torch.Size([1, 10, 7])
예측된 클래스 ID: tensor([[1, 2, 2, 2, 2, 2, 2, 2, 1, 1]], device='cuda:0')


4. 예측된 가중츠니느 랜덤한 값이므로, 모델을 미세 튜닝하기 전까지는 신뢰할 수 없다.

In [20]:
# 사전 훈련된 모뎅의 예측 겨로가를 태그 이름으로 바꿔 토큰과 함께 확인한다.
preds = [tags.names[pred] for pred in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["토큰", "예측된 태그"])

,0,1,2,3,4,5,6,7,8,9
토큰,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
예측된 태그,B-PER,I-PER,I-PER,I-PER,I-PER,I-PER,I-PER,I-PER,B-PER,B-PER


In [21]:
# 헬퍼 함수 작성
def tag_text(text, tags, model, tokenizer):
    # 텍스트를 토큰으로 분할하고 입력 ID로 인코딩한다.
    tokens = tokenizer(text).tokens()
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    
    # 가능한 일곱 개의 클래스에 대한 로짓을 출력한다.
    outputs = model(input_ids)[0]
    
    # 각 토큰에 대해 확률이 가장 높은 클래스 ID를 선택한다.
    predictions = outputs.argmax(dim=2)
    
    # 예측된 클래스 ID를 태그 이름으로 변환한다.
    preds = [tags.names[pred] for pred in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["토큰", "예측된 태그"])

In [ ]:
print(tag_text("Barack Obama was born in Hawaii.", tags, xlmr_model, xlmr_tokenizer))